### 1_Science_Technology_(PCA)

### Import

In [81]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from pyclustertend import hopkins
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

plt.style.use('classic')
sns.set_style("white")

### Data Scraping Oecd

In [82]:
# import pandas as pd
# import pandasdmx as sdmx
# import pandas_datareader.data as web

# pd.set_option('display.max_rows', None)#print everything rows
# pd.set_option('display.max_columns', None)#print everything columns

# series = 'STIO_2016'
# source = 'oecd'
# start = '2006-01-01'
# end = '2016-01-01'

# oecd = sdmx.Request('OECD')
# params = dict(startPeriod=start, endPeriod=end)
# data_msg = oecd.data('STIO_2016', key='all', params=params)
# data = data_msg.data[0]
# df = sdmx.to_pandas(data).unstack() #turn data into a dataframe
# print(df)

In [83]:
#df.to_csv("science_technology.csv")

### Data Info

In [84]:
df = pd.read_csv("science_technology.csv")
df.head()

,COUNTRY,INDICATOR,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,ARG,ADTERPOP_XT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARG,BE_AF_NC,9.80,9.90,11.80,12.10,16.50,23.39,27.41,32.94,NaN,NaN
2,ARG,BE_AF_PPP,7.29,6.42,6.44,6.05,7.11,8.78,8.87,9.15,NaN,NaN
3,ARG,BE_AF_PPPK,7.76,6.66,6.55,6.11,7.11,8.55,8.45,8.48,NaN,NaN
4,ARG,BE_AF_XGDP,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN


In [85]:
df=pd.melt(df,
           id_vars=["COUNTRY","INDICATOR"],
           value_vars=["2006","2007","2008","2009","2010","2011","2012","2013","2014","2015"],
           var_name="TIME",
           value_name="VALUE")
df.head(2)

,COUNTRY,INDICATOR,TIME,VALUE
0,ARG,ADTERPOP_XT,2006,NaN
1,ARG,BE_AF_NC,2006,9.8


In [86]:
df = df.pivot(index=["COUNTRY","TIME"], columns="INDICATOR", values=["VALUE"])
df.columns = df.columns.droplevel()
df = df.rename_axis(None, axis=1)
df = df.reset_index()
df.head(1)

,COUNTRY,TIME,ABS_I,ADTERPOP_XT,AFA_XBE,AGEPOP_MEDIAN,BE_AF_NC,BE_AF_PPP,BE_AF_PPPK,BE_AF_XGDP,...,WUGTARP_SCI_NB,WUGTARP_XT,WUGTA_ENG_NB,WUGTA_NB,WUGTA_SCI_NB,WUGTA_XT,WUNIVG_TERABARP_NB,WUNIVG_TERABARP_XT,XBR_I,YDEP_X1564
0,ARG,2006,NaN,NaN,NaN,NaN,9.8,7.29,7.76,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df.shape

(510, 499)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Columns: 499 entries, COUNTRY to YDEP_X1564
dtypes: float64(497), object(2)
memory usage: 1.9+ MB


In [89]:
df.describe()

,ABS_I,ADTERPOP_XT,AFA_XBE,AGEPOP_MEDIAN,BE_AF_NC,BE_AF_PPP,BE_AF_PPPK,BE_AF_XGDP,BE_AR_NC,BE_AR_PPP,...,WUGTARP_SCI_NB,WUGTARP_XT,WUGTA_ENG_NB,WUGTA_NB,WUGTA_SCI_NB,WUGTA_XT,WUNIVG_TERABARP_NB,WUNIVG_TERABARP_XT,XBR_I,YDEP_X1564
count,80.000000,296.000000,148.000000,2.000000,283.000000,289.000000,283.000000,259.000000,2.050000e+02,218.000000,...,259.000000,135.000000,5.000000,11.000000,5.0000,6.000000,6.000000,6.000000,80.000000,457.000000
mean,2.119250,25.004831,33.942973,36.755000,8523.961943,1424.973737,1130.361025,0.121081,3.661786e+05,3776.514587,...,9230.308456,31.070148,1985.200000,139403.454545,3212.6000,59.808333,278841.166667,54.903333,0.534875,28.089453
std,0.787815,10.628215,16.499657,1.746554,23831.856300,3026.114870,2280.036939,0.246690,1.316399e+06,6301.574545,...,16424.962694,7.764802,1362.329329,94301.779364,3435.2389,1.048283,28301.040217,2.957713,0.688796,8.003298
min,0.840000,4.170000,2.540000,35.520000,4.520000,0.040000,0.050000,0.000000,7.980000e+00,14.420000,...,0.000000,15.460000,920.000000,21236.000000,1229.0000,58.460000,237823.000000,49.150000,0.010000,19.050000
25%,1.530000,17.282500,24.157500,36.137500,160.830000,77.050000,71.980000,0.020000,4.448000e+02,338.040000,...,1424.000000,25.775000,1301.000000,28285.000000,1589.0000,59.055000,258825.250000,54.977500,0.117500,21.850000
50%,2.085000,24.945000,31.610000,36.755000,1223.900000,362.600000,349.370000,0.060000,3.302370e+03,767.645000,...,2606.000000,31.430000,1451.000000,194194.000000,1600.0000,59.770000,286694.500000,55.690000,0.200000,25.730000
75%,2.465000,32.365000,41.760000,37.372500,3952.550000,1388.330000,1341.680000,0.130000,8.786380e+03,3995.427500,...,13327.500000,36.110000,1917.000000,212890.500000,2329.0000,60.755000,302463.250000,56.725000,0.505000,30.560000
max,4.750000,61.930000,72.370000,37.990000,282719.000000,19347.000000,18314.240000,1.790000,8.479291e+06,34021.440000,...,108537.000000,52.780000,4337.000000,242980.000000,9316.0000,60.950000,304580.000000,57.080000,2.900000,50.260000


In [90]:
df.isna().any()

COUNTRY               False
TIME                  False
ABS_I                  True
ADTERPOP_XT            True
AFA_XBE                True
                      ...  
WUGTA_XT               True
WUNIVG_TERABARP_NB     True
WUNIVG_TERABARP_XT     True
XBR_I                  True
YDEP_X1564             True
Length: 499, dtype: bool

In [95]:
# Find the over %?-NAN-Value-Features and Drop them
drop_list = []
remain_list = []

for i in df.columns[2:]:
    a = list(df[i].isna().value_counts())[0]
    b = len(df[i])
    c = (a / b) * 100
    if c > 90:
        drop_list.append(i)
    else:
        remain_list.append(i)

In [96]:
print(len(drop_list))
print(len(remain_list))

70
427
